In [1]:
import os
dataset_path = "Monkeypox Skin Image Dataset"

class_names = os.listdir(dataset_path)
print("Class names:", class_names)

Class names: ['Chickenpox', 'Measles', 'Monkeypox', 'Normal']


In [2]:
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)

    if os.path.isdir(class_path):
        count = 0
        for img in os.listdir(class_path):
            count = count + 1

        print(class_name, ":", count, "images")

Chickenpox : 91 images
Measles : 91 images
Monkeypox : 91 images
Normal : 91 images


In [5]:
# Split the dataset into three parameters like train, validation and test for building the model
# save it into output folder
import random
import shutil

output_path = "E:\MaxGen\git2025\MPDD\MonkeyPox-Disease-Prediction-System\dataset_split"

train_ratio = 0.7
test_ratio = 0.1
val_ratio = 0.2

for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)


    if os.path.isdir(class_path):

        images = os.listdir(class_path)
        random.shuffle(images)

        total = len(images)
        train_end = int(total * train_ratio)
        val_end = train_end + int(total * val_ratio)

        train_images = images[:train_end]
        val_images = images[train_end:val_end]
        test_images = images[val_end:]


        for folder in ['train', 'val', 'test']:
            os.makedirs(os.path.join(output_path, folder, class_name), exist_ok=True)

        for img in train_images:
            shutil.copy(
                os.path.join(class_path, img),
                os.path.join(output_path, 'train', class_name, img)
                
            )

        for img in test_images:
            shutil.copy(
                os.path.join(class_path, img),
                os.path.join(output_path, 'val', class_name, img)
            )

        for img in test_images:
            shutil.copy(
                os.path.join(class_path, img),
                os.path.join(output_path, 'test', class_name, img)
            )

        print(class_name, "-> Train: ", len(train_images),
              "Val:", len(val_images),
              "Test:", len(test_images)
             )

Chickenpox -> Train:  63 Val: 18 Test: 10
Measles -> Train:  63 Val: 18 Test: 10
Monkeypox -> Train:  63 Val: 18 Test: 10
Normal -> Train:  63 Val: 18 Test: 10


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/kaggle/working/output_dataset/train'
val_dir = '/kaggle/working/output_dataset/val'
test_dir = '/kaggle/working/output_dataset/test'

img_size = (224,224)
batch_size = 32